# Reto 2: Indexando palabras de archivos de texto

## Descripción
Crear el script `grepw.py` que dada una `palabra` y una `carpeta` imprima la lista de todos los archivos de texto que contengan la `palabra`. Se sugiere usar el resultado del **Reto-1** para obtener la lista de archivos y entonces se puede resolver de dos maneras:

- **Soluciòn 1:** Ya con la lista de archivos de tipo texto iterar sobre la lista leyendo el contenido de cada archivo y validando si la `palabra` existe en el archivo en turno, si si, entonces se agrega el archivo a la lista resultado, si no se omite el archivo, al final se imprime la lista resultado.
- **Solución 2:** Ya con la lista de archivos de texto, iterar sobre la lista leyendo el contenido de cada archivo, obteniendo la lista de palabras contenidas en cada archivo y almacenando el resultado en una base de datos con las siguientes tablas:

| Palabras |
| -------- |
| id INT   |
| palabra VARCHAR(20) |

| Archivos |
| -------- |
| id INT   |
| nombre VARCHAR(1024) |

| PalabrasArchivo |
| -------- |
| id INT   |
| id_palabra INT |
| id_archivo INT |

De tal forma que cada palabra encontrada en el archivo en turno se agrega a la tabla `Palabras` siempre y cuando la palabra no exista y el nombre del archivo en la tabla `Archivos` y luego por cada palabra en el archivo se agrega un registro en la tabla `PalabrasArchivo` por ejemplo si el archivo `hola-mundo.txt` tiene el texto `Hola mundo de Python` entonces el contenido de las tablas sería el siguiente:

| Palabras | |
| -------- | - |
| 1   | hola |
| 2   | mundo |
| 3   | de |
| 4   | python |

| Archivos | |
| -------- | - |
| 1   | hola-mundo.txt |

| PalabrasArchivo | | |
| -------- | - | - |
| 1   | 1 | 1 |
| 2   | 2 | 1 |
| 3   | 3 | 1 |
| 4   | 4 | 1 |

Así que si se busca la lista de archivos que contienen por ejemplo la palabra `mundo` entonces la consulta en SQL sería:

```sql
SELECT Archivos.nombre FROM Palabras
LEFT JOIN PalabrasArchivo ON Palabras.id=PalabrasArchivo.id_palabra
LEFT JOIN Archivos ON PalabrasArchivo.id_archivo=Archivos.id
WHERE palabra="mundo";
```

El resultado de la consulta se almacena en una lista de resultado y se imprime.

Si la base de datos ya existe entonces sólo se hace la consulta y ya no es necesario construir la base de datos, en otras palabras no es necesario indexar los archivos de la carpeta nuevamente.

**Nota:** Aparentemente la solución 2 puede tardar más tiempo al realizar la primer consulta porque se tiene que construir la base de datos, pero en las siguiente consultas el resultado se debería de obtener en segundos.

Por ejemplo si se tuviera unas carpetas con los siguientes archivos:

```
carpeta/
|- archivo1.txt
|- archivo2.txt
|- archivo3.exe
|- archivo4.xlsx
|- subcarpeta/
   |- archivo5.html
   |- archivo6.jpg
   |- archivo7.png
   |- archivo8.csv
```
A continuación el resultado de buscar algunas palabras:
```bash
$ python grepw.py Juan carpeta
carpeta/subcarpeta/archivo8.csv
$ python grepw.py para carpeta
carpeta/archivo1.txt
carpeta/subcarpeta/archivo5.html
$
```

## Desarrollo

Como se va a usar la función de `obtener_archivos_texto()` del **Reto-1** entonces la parte principal del programa muy similar, así que comenzamos imprimiendo la lista de archivos de texto:

In [78]:
import lst
import pathlib

# Se obtiene carpeta desde el usuario
ruta_str = "carpeta"
ruta = pathlib.Path(ruta_str)
palabra = "para"

# Se obtiene la lista de archivos a partir de ruta
archivos = lst.obtener_archivos_texto(ruta)  # La función existe en el módulo lst

# Se imprime la lista
for nom in archivos:
    print(nom)

carpeta/archivo1.txt
carpeta/archivo2.txt
carpeta/subcarpeta/archivo8.csv
carpeta/subcarpeta/archivo5.html


La celda a continuación tiene ya parte de la solución, sin embargo siempre regresará una lista vacía porque la función `archivos_con_palabra()` no está terminada.

Tu **Reto** consiste en terminar la función `archivos_con_palabras()` que debe realizar una consulta a la base de datos, el resultado se puede obtener por varios caminos, sin embargo se tiene que hacer uso de las tres tablas, ya sea que realicen las consultas por separado o también puedes hacer uso de las instrucción LEFT JOIN de SQL.

Para validar que el código funciona deberás de poder obtener los resultados que se muestran en las descripción del reto para la carpeta `carpeta`.

Cuando hayas terminado pasa a la siguiente celda.

In [79]:
import sqlite3
import string

def crea_tablas(conn):
    """ Crea la tablas para la base de datos de indexación de palabras """
    cur = conn.cursor()
    sql = """CREATE TABLE Palabras (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    palabra VARCHAR(20)
)"""
    cur.execute(sql)
    sql = """CREATE TABLE Archivos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre VARCHAR(1024)
)"""
    cur.execute(sql)
    sql = """CREATE TABLE PalabrasArchivo (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    id_palabra INTEGER REFERENCES Palabras(id),
    id_archivo INTEGER REFERENCES Palabras(id)
)"""
    cur.execute(sql)
    conn.commit()
    cur.close()

def inserta_registros(conn, archivos):
    """ Itera sobre la lista de archivos de texto obteniendo la lista de
    palabras agregando cada una a la tabla Palabras, el nombre del archivo
    a la tabla Archivos y tantas registros a la tabla PalabrasArchivo como
    palabras existan en el archivo """
    cur = conn.cursor()
    for patharch in archivos:
        sql = "INSERT INTO Archivos (nombre) VALUES (?)"
        cur.execute(sql, (patharch.as_posix(),))
        arch_id = cur.lastrowid
        # print("Archivos INSERT ->", arch_id, patharch)
        # conn.commit()
        texto = patharch.read_text()
        mapa_puntuacion = str.maketrans(string.punctuation, " "*len(string.punctuation))
        texto_palabras = texto.translate(mapa_puntuacion)
        lista_palabras = texto_palabras.split()
        lista_limpia = [palabra.strip() for palabra in lista_palabras]
        lista_sin_repetidas = set(lista_limpia)
        for palabra in lista_sin_repetidas:
            sql = "SELECT * FROM Palabras WHERE palabra=?"
            cur.execute(sql, (palabra,))
            tabla_palabras = cur.fetchall()
            if len(tabla_palabras) == 1:
                palabra_id = tabla_palabras[0][0]
            else:
                sql = "INSERT INTO Palabras (palabra) VALUES (?)"
                cur.execute(sql, (palabra,))
                palabra_id = cur.lastrowid
                # print("Palabras INSERT ->", palabra_id, palabra)

            sql = "INSERT INTO PalabrasArchivo (id_palabra, id_archivo) VALUES (?, ?)"
            cur.execute(sql, (palabra_id, arch_id))
            # print("PalabrasArchivo INSERT ->", palabra_id, arch_id)
        conn.commit()
    cur.close()

#############################
### Complementa esta función
#############################
def archivos_con_palabra(conn, palabra):
    """ Busca en la base de datos todos los archivos que contengan palabra
    y regresa una lista de cadenas con los archivos encontrados

    conn - Es la variable de la conexión a la base de datos
    palabra - Es la palabra a buscar
    """
    lista_archivos = []
    return lista_archivos

# Si no existe la base de datos se crea
ruta_bd_str = ".index.sqlite3"
ruta_bd = ruta.joinpath(ruta_bd_str)
if not ruta_bd.exists():
    conn = sqlite3.connect(ruta_bd.as_posix())
    crea_tablas(conn)
    inserta_registros(conn, archivos)
    conn.close()

# Si la base de datos existe se abre para realizar la consulta de buscar la palabra
conn = sqlite3.connect(ruta_bd.as_posix())
archivos_resultado = archivos_con_palabra(conn, palabra)  ### La variable palabra está definida en la primer celda de código
conn.close()

# Se imprime la lista resultado
for nom in archivos_resultado:
    print(nom)

carpeta/archivo1.txt
carpeta/subcarpeta/archivo5.html


### Creando el script `grepw.py`

Si has llegado hasta aquí significa que has hecho un buen trabajo y ahora tendrás que crear el script `grepw.py` por lo que necesitas crear el archivo con JupyterLab y copiar el código de las celdas de código (realizalo similar a como se creó el script `lst.py`, incluso puedes copiar parte del código también si te es de utilidad)

Recuerda que la sintaxis o forma de usar del script es:

`python grepw.py palabra carpeta`

Donde:

**palabra** es la palabra a buscar en los archivos
**carpeta** la carpeta donde se búscarán los archivos de texto y en ellos la palabra

Ejecutar el script hasta obtener el resultado mostrado en las descripción.